In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
from tqdm import tqdm_notebook

from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from collections import Counter
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 추천시스템
import surprise
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic

In [2]:
data = pd.read_excel('siksin_2.xlsx')

In [3]:
data

,Unnamed: 0,닉네임,평가,평점,가게명,지역
0,273,!날아라슈퍼맨!,매운족발 그냥 깔끔합니다.,3.0,꿀벅지족발,"서울-강북, 남산/후암동"
1,976,!날아라슈퍼맨!,한우 갈비살 양념구이 먹엇는데 석쇠에 빨리빨리 구워서 먹는데 여기특이한게 고추장에 ...,4.0,영양숯불갈비,"경북, 경주시"
2,2438,!날아라슈퍼맨!,닭불고기 정말 만족해요,3.0,무등산닭불고기,"광주, 서구-금호/풍암"
3,3341,!날아라슈퍼맨!,제주도에서 먹은 고기국수 만큼 괜찮았어요 ㅎㅎ,3.0,아랑졸디,"경기, 구리"
4,3467,!날아라슈퍼맨!,누룽지백숙!! 백숙국물에 누룽지를 넣어서 나오는데 담백하고 고소하고 닭백숙이 되게 ...,3.0,산촌,"경기, 의왕/백운호수"
...,...,...,...,...,...,...
5340,6024,힙통령,만족스러워요,2.5,내림손삼계탕,"서울-강북, 명동"
5341,628,힙합간지녀,돼지갈비도 인기 많아요,3.0,청석정육식당,"경기, 광주시"
5342,767,힙합간지녀,등심 진짜 맛있어요,3.0,창신식육식당,"경북, 영주"
5343,2214,힙합간지녀,고기를 직접 골라서 먹을 수 있는게 정말 맘에 들어요 ㅋㅋ 한우랑 키조개랑 같이 먹...,3.0,탐마루장흥한우,"전남, 장흥"


In [4]:
print(data.shape)

(5345, 6)


In [5]:
df = data[['닉네임', '가게명', '평점']]
df['평점'] = df['평점'].astype(int)
df

,닉네임,가게명,평점
0,!날아라슈퍼맨!,꿀벅지족발,3
1,!날아라슈퍼맨!,영양숯불갈비,4
2,!날아라슈퍼맨!,무등산닭불고기,3
3,!날아라슈퍼맨!,아랑졸디,3
4,!날아라슈퍼맨!,산촌,3
...,...,...,...
5340,힙통령,내림손삼계탕,2
5341,힙합간지녀,청석정육식당,3
5342,힙합간지녀,창신식육식당,3
5343,힙합간지녀,탐마루장흥한우,3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345 entries, 0 to 5344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     5345 non-null   object
 1   가게명     5345 non-null   object
 2   평점      5345 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 104.5+ KB


In [7]:
# 판다스를 딕셔너리로 만드는 함수
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d

In [8]:
name_list = []      # 사용자 목록을 담을 리스트   # 중복 가능
food_set = set()     # 화장품 목록을 담을 set      # 중복 불가

In [9]:
df_dict = recur_dictify(df)

In [10]:
# 사용자 수 만큼 반복한다.
for user_key in df_dict:  # user_key : 각각의 유저이름
#     print(user_key)
    name_list.append(user_key)
    
    # 현재 사용자가 본 화장품 목록을 set에 담는다.
    for food_key in df_dict[user_key] :   # 각각의 유저이름에 해당하는 각각의 제품이름
        # print(user_key, ":", cos_key)
        food_set.add(food_key)  

In [11]:
food_set

{'우리보리생고기',
 '조선본갈비',
 '대원갈비',
 '중앙찜닭',
 '훈남닭발 창원상남점',
 '최씨부엌',
 '수정닭갈비',
 '진미언양불고기',
 '원조춘천닭갈비',
 '샘토참숯구이',
 '낭만닭발',
 '바로화덕치킨',
 '성미가든',
 '명동일번지닭갈비',
 '원평지평선청보리한우촌',
 '고기의 잔치',
 '애플삼겹살 선릉점',
 '삼목등심',
 '느티나무집',
 '원할머니 보쌈족발 창원상남점',
 '남도고기국수',
 '터갈비',
 '돈바른 수제갈비',
 '돌판삼겹과 양푼찌개',
 '신마포갈매기',
 '날잡아잡소갈비집',
 '동대구식당',
 '동대문곱창',
 '영도숯불갈비',
 '총각네부추곱창',
 '춘천호반닭갈비',
 '운제산미나리삼겹살',
 '할매곱창',
 '장안사 산장',
 '윤가네한우마을',
 '홍천족발',
 '서초갈비',
 '산동관',
 '육병장(성건점)',
 '한우뭉치',
 '대한뉴스',
 '물갯말닭갈비',
 '천하불닭발',
 '하루방연탄집',
 '최고의 한우 명우',
 '최씨부엌족발 봉선점',
 '투뿔등심 (강남역점)',
 '고양한우마을',
 '일자르디노',
 '화수목',
 '부영 담양식 숯불갈비',
 '춘천통나무집닭갈비',
 '이레옥',
 '삼고집 춘천점',
 '60년대식소금구이',
 '동산갈비',
 '한가네닭발',
 '꿀래',
 '동광식당',
 '하얀집가든',
 '연지갈비',
 '히노케 서면점',
 '곱창폭식',
 '서울불고기',
 '정가네정육식당',
 '경복궁불고기 송도점',
 '우왕',
 '포도식당',
 '천년의 비빔밥',
 '모다기',
 '존맛불고기',
 '한방왕족발',
 '흥부족발보쌈',
 '한국왕족발보쌈',
 '박가네집',
 '압구정한우갈비',
 '부추곱창',
 '쌍둥이네서서갈비',
 '참숯닭갈비반',
 '새빨간죠스찜닭 경북경주노동점',
 '설악한우마을',
 '개군한우',
 '돈주는남자',
 '갈비1987',
 '서서곱창',
 '낙원식당',
 '횡성토종한우프라자',
 '시장닭발',
 '맛찬들왕소금구이 서면점',
 '김덕후의

In [12]:
food_list = list(food_set)

print(len(food_list))
food_list

1982


['우리보리생고기',
 '조선본갈비',
 '대원갈비',
 '중앙찜닭',
 '훈남닭발 창원상남점',
 '최씨부엌',
 '수정닭갈비',
 '진미언양불고기',
 '원조춘천닭갈비',
 '샘토참숯구이',
 '낭만닭발',
 '바로화덕치킨',
 '성미가든',
 '명동일번지닭갈비',
 '원평지평선청보리한우촌',
 '고기의 잔치',
 '애플삼겹살 선릉점',
 '삼목등심',
 '느티나무집',
 '원할머니 보쌈족발 창원상남점',
 '남도고기국수',
 '터갈비',
 '돈바른 수제갈비',
 '돌판삼겹과 양푼찌개',
 '신마포갈매기',
 '날잡아잡소갈비집',
 '동대구식당',
 '동대문곱창',
 '영도숯불갈비',
 '총각네부추곱창',
 '춘천호반닭갈비',
 '운제산미나리삼겹살',
 '할매곱창',
 '장안사 산장',
 '윤가네한우마을',
 '홍천족발',
 '서초갈비',
 '산동관',
 '육병장(성건점)',
 '한우뭉치',
 '대한뉴스',
 '물갯말닭갈비',
 '천하불닭발',
 '하루방연탄집',
 '최고의 한우 명우',
 '최씨부엌족발 봉선점',
 '투뿔등심 (강남역점)',
 '고양한우마을',
 '일자르디노',
 '화수목',
 '부영 담양식 숯불갈비',
 '춘천통나무집닭갈비',
 '이레옥',
 '삼고집 춘천점',
 '60년대식소금구이',
 '동산갈비',
 '한가네닭발',
 '꿀래',
 '동광식당',
 '하얀집가든',
 '연지갈비',
 '히노케 서면점',
 '곱창폭식',
 '서울불고기',
 '정가네정육식당',
 '경복궁불고기 송도점',
 '우왕',
 '포도식당',
 '천년의 비빔밥',
 '모다기',
 '존맛불고기',
 '한방왕족발',
 '흥부족발보쌈',
 '한국왕족발보쌈',
 '박가네집',
 '압구정한우갈비',
 '부추곱창',
 '쌍둥이네서서갈비',
 '참숯닭갈비반',
 '새빨간죠스찜닭 경북경주노동점',
 '설악한우마을',
 '개군한우',
 '돈주는남자',
 '갈비1987',
 '서서곱창',
 '낙원식당',
 '횡성토종한우프라자',
 '시장닭발',
 '맛찬들왕소금구이 서면점',
 '김덕후의

In [13]:
# 학습할 데이터를 준비한다.
rating_dic = {
    '닉네임' : [],
    '가게명' : [],
    '평점' : []
    }

In [14]:
rating_dic

{'닉네임': [], '가게명': [], '평점': []}

In [15]:
# user 수만큼 반복
user_list = []
for user_key in df_dict :  # user_key : 각각의 유저이름
    user_list.append(user_key)
    # 해당 사용자가 본 화장품 수 만큼 반복
    for food_key in df_dict[user_key] :

        # 사용자 인덱스 번호를 추출한다.
        a1 = name_list.index(user_key)

        # 화장품 인덱스 번호를 추출한다.
        a2 = food_list.index(food_key)

        # 평점을 가져온다.
        a3 = df_dict[user_key][food_key]

        # 딕셔너리에 담는다.
        rating_dic['닉네임'].append(a1)
        rating_dic['가게명'].append(a2)
        rating_dic['평점'].append(a3)

In [16]:
rating_dic

{'닉네임': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  8,
  8,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  10,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  12,
  12,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  14,
  14,
  14,
  14,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  18,
  18,
  18,
  19,
  19,
  19,
  20,
  20,
  20,
  20,
  20,
  21,
  22,
  22,
  23,
  23,
  23,
  24,
  24,
  25,
  25,
  25,
  25,
  25,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  27,
  27,
  27,
  27,
  28,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  32,
  32,
  32,
  32,
  33,
  33,
  33,
  3

In [17]:
df_index = pd.DataFrame(rating_dic)
df_index

,닉네임,가게명,평점
0,0,1412,3
1,0,144,3
2,0,970,3
3,0,660,3
4,0,566,3
...,...,...,...
5340,928,214,3
5341,929,1106,3
5342,929,461,3
5343,929,840,3


In [18]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345 entries, 0 to 5344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   닉네임     5345 non-null   int64
 1   가게명     5345 non-null   int64
 2   평점      5345 non-null   int64
dtypes: int64(3)
memory usage: 125.4 KB


In [19]:
# pivot_table 메소드를 사용해서 행렬 변환 (가게명과 닉네임 활용)
ratings_matrix = df.pivot_table('평점', index='닉네임', columns='가게명')

print(ratings_matrix.shape)
ratings_matrix

# pivot_table 메소드를 사용해서 행렬 변환
#ratings_matrix = df_index.pivot_table('평점', index='닉네임', columns='가게명')

#print(ratings_matrix.shape)
#ratings_matrix


(930, 1982)


가게명,1.5닭갈비 본점,101닭갈비,28치킨,2번가포차,37Don,401레스토랑,5.5 닭갈비,5.5닭갈비 본점,55온족발,5k치킨 선릉본점,...,훈이네,훌랄라치킨카페&천족,흑돈연가,흥부가한우돌곱창,흥부골이동왕숯불갈비,흥부놀부족발,흥부보쌈,흥부족발보쌈,히노케 서면점,히로아키
닉네임,,,,,,,,,,,,,,,,,,,,,
!날아라슈퍼맨!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#Dali_Mom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#_허니잼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#돼랑이라그램,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#먹방,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황금돼지,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
후스콜미010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
휴먼졸림체,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

가게명,1.5닭갈비 본점,101닭갈비,28치킨,2번가포차,37Don,401레스토랑,5.5 닭갈비,5.5닭갈비 본점,55온족발,5k치킨 선릉본점,...,훈이네,훌랄라치킨카페&천족,흑돈연가,흥부가한우돌곱창,흥부골이동왕숯불갈비,흥부놀부족발,흥부보쌈,흥부족발보쌈,히노케 서면점,히로아키
닉네임,,,,,,,,,,,,,,,,,,,,,
!날아라슈퍼맨!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#Dali_Mom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#_허니잼,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#돼랑이라그램,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#먹방,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황금돼지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
후스콜미010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
휴먼졸림체,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()  # 전치 행렬

print(ratings_matrix_T.shape)
ratings_matrix_T.head(3)

(1982, 930)


닉네임,!날아라슈퍼맨!,#Dali_Mom,#_허니잼,#돼랑이라그램,#먹방,#혼자밥,*모래알은반짝*,*뽀로로*,104731,10시25분,...,호랑이기운이쑥쑥,호호아가씨,홍기선,화장하는금자씨,화장해야예뽀,황금돼지,후스콜미010,휴먼졸림체,힙통령,힙합간지녀
가게명,,,,,,,,,,,,,,,,,,,,,
1.5닭갈비 본점,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101닭갈비,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28치킨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# 가게와 가게들 간 코사인 유사도 산출
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(1982, 1982)


가게명,1.5닭갈비 본점,101닭갈비,28치킨,2번가포차,37Don,401레스토랑,5.5 닭갈비,5.5닭갈비 본점,55온족발,5k치킨 선릉본점,...,훈이네,훌랄라치킨카페&천족,흑돈연가,흥부가한우돌곱창,흥부골이동왕숯불갈비,흥부놀부족발,흥부보쌈,흥부족발보쌈,히노케 서면점,히로아키
가게명,,,,,,,,,,,,,,,,,,,,,
1.5닭갈비 본점,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101닭갈비,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28치킨,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# 곱창폭식과 유사한 가게 6개 확인해보기
#item_sim_df[164].sort_values(ascending=False)[:6]
# 가게명만 곱창폭식으로 변경
item_sim_df["곱창폭식"].sort_values(ascending=False)[:6]

가게명
남촌        1.000000
진미불고기     1.000000
마굿간생고기    1.000000
미담진족      0.707107
꼴통치킨      0.639602
대평갈비      0.577350
Name: 남촌, dtype: float64

In [225]:
# 자기 것 빼고 곱창폭식 유사한 가게 5개 확인해보기
#item_sim_df[164].sort_values(ascending=False)[1:6]
#가게명만 곱창폭식으로 변경
item_sim_df["곱창폭식"].sort_values(ascending=False)[1:6]

가게명
강릉풍년갈비      0.832050
천사곱창 화곡점    0.832050
청계마루        0.588348
육대감         0.554700
수뿌레닭갈비      0.532181
Name: 곱창폭식, dtype: float64

인접 이웃 협업 필터링으로 개인화된 고깃집 추천

In [26]:
# 평점 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 평점을 계산하는 함수 정의
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [27]:
item_sim_df.head(3)

가게명,1.5닭갈비 본점,101닭갈비,28치킨,2번가포차,37Don,401레스토랑,5.5 닭갈비,5.5닭갈비 본점,55온족발,5k치킨 선릉본점,...,훈이네,훌랄라치킨카페&천족,흑돈연가,흥부가한우돌곱창,흥부골이동왕숯불갈비,흥부놀부족발,흥부보쌈,흥부족발보쌈,히노케 서면점,히로아키
가게명,,,,,,,,,,,,,,,,,,,,,
1.5닭갈비 본점,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101닭갈비,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28치킨,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11498357],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.06416751, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [29]:
# 데이터프레임으로 변환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(10)

(930, 1982)


가게명,1.5닭갈비 본점,101닭갈비,28치킨,2번가포차,37Don,401레스토랑,5.5 닭갈비,5.5닭갈비 본점,55온족발,5k치킨 선릉본점,...,훈이네,훌랄라치킨카페&천족,흑돈연가,흥부가한우돌곱창,흥부골이동왕숯불갈비,흥부놀부족발,흥부보쌈,흥부족발보쌈,히노케 서면점,히로아키
닉네임,,,,,,,,,,,,,,,,,,,,,
!날아라슈퍼맨!,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.114984
#Dali_Mom,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.053696,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
#_허니잼,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.039248,0.042139,0.000000,...,0.061264,0.000000,0.0,0.038442,0.0,0.0,0.000000,0.000000,0.0,0.000000
#돼랑이라그램,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.065707,0.298256,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.085921,0.000000,0.0,0.000000
#먹방,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
#혼자밥,0.0,0.0,0.339448,0.0,0.0,0.0,0.000000,0.000000,0.040649,0.000000,...,0.052974,0.000000,0.0,0.032563,0.0,0.0,0.021089,0.000000,0.0,2.687236
*모래알은반짝*,0.0,0.0,0.000000,0.0,0.0,0.0,0.057077,0.000000,0.000000,0.000000,...,0.000000,0.075938,0.0,0.073591,0.0,0.0,0.024400,0.087908,0.0,0.000000
*뽀로로*,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.044785,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
104731,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000


예측 평점 정확도 RMSE

In [30]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  4.652098525513761


Top-n 유사도를 가진 데이터들에서만 예측 평점 계산

In [33]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred


In [34]:
# 실행시간 2분 정도 걸림
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  2.2244744120722912


In [235]:
# 사용자 9번에게 고깃집을 추천해보자
# 추천에 앞서 9번 사용자가 높은 평점을 준 가게를 확인해보면
#user_rating_id = ratings_matrix.loc[9, :]
#사용자 이름만 변경상
user_rating_id = ratings_matrix.loc["10시25분", :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

가게명
돌다리보쌈           4.0
항구치킨cafe        3.0
하늘가득장수한우전문점     3.0
파절이삼겹살          3.0
칼맞은 삼겹살(강남점)    3.0
우리식당            3.0
오백집 모자족발        3.0
예산축협한우명품관       3.0
어화장단            3.0
백두한우곱창          3.0
Name: 10시25분, dtype: float64

자기가 안간 가게 중에 추천

In [231]:
def get_didnt_go(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 가게정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 가게명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 갔던 가게임. 대상 index를 추출하여 list 객체로 만듬
    already_gone = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 가게명을 list 객체로 만듬. 
    stores_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_gone에 해당하는 가게는 제외함. 
    didnt_list = [ store for store in stores_list if store not in already_gone]
    
    return didnt_list

In [232]:
# pred_df : 앞서 계산된 가게 별 예측 평점
# didnt_list : 사용자가 가지 않은 가게들
# top_n : 상위 n개를 가져온다.

def recomm_store_by_userid(pred_df, userId, didnt_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 가게명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_stores = pred_df.loc[userId, didnt_list].sort_values(ascending=False)[:top_n]
    return recomm_stores

In [234]:
# 사용자가 가지 않은 가게명 추출   
#didnt_list = get_didnt_go(ratings_matrix, 9)
didnt_list = get_didnt_go(ratings_matrix, "10시25분")

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
#recomm_stores = recomm_store_by_userid(ratings_pred_matrix, 9, didnt_list, top_n=10)
recomm_stores = recomm_store_by_userid(ratings_pred_matrix, "10시25분", didnt_list, top_n=10)
# 평점 데이타를 DataFrame으로 생성. 
recomm_stores = pd.DataFrame(data=recomm_stores.values,index=recomm_stores.index, columns=['pred_score'])
recomm_stores

,pred_score
가게명,
동산갈비,0.660123
한잔육회,0.454366
이귀남찜닭,0.438800
김덕후곱창조,0.354367
돈원 묵은지 김치찌개,0.342477
홍씨네닭갈비,0.342477
산야초갈비이야기,0.342477
백조횟집,0.342477
청담,0.339624
